In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
d2r = lambda x: x * np.pi / 180 # convert degrees to radians
def getDistance(lat1, lon1, lat2, lon2): 
    """Calculate the distance between two (lat,lon) points using 'Haversine' Formula"""
    r = 6371 # radias of earth in km
    dLat = d2r(lat2 - lat1)
    dLon = d2r(lon2 - lon1)
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(d2r(lat1)) \
         * math.cos(d2r(lat2)) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = r * c # Distance in km
    return d

In [3]:
gold = pd.read_csv('gold_reserves.csv', header=0)
cities = pd.read_csv('country-capitals.csv', header=0)

In [4]:
gold.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,Unnamed: 61
0,Aruba,ABW,"Total reserves (includes gold, current US$)",FI.RES.TOTL.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,6.918739e+08,6.869701e+08,7.087137e+08,6.897760e+08,7.686223e+08,6.531707e+08,6.803510e+08,8.158006e+08,9.225509e+08,NaN
1,Afghanistan,AFG,"Total reserves (includes gold, current US$)",FI.RES.TOTL.CD,5.069080e+07,42444500.0,4.059210e+07,4.554780e+07,4.461968e+07,4.543456e+07,...,3.042274e+09,4.265889e+09,5.162440e+09,6.344642e+09,7.152304e+09,7.288703e+09,7.528550e+09,6.976966e+09,7.281910e+09,NaN
2,Angola,AGO,"Total reserves (includes gold, current US$)",FI.RES.TOTL.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.786941e+10,1.366410e+10,1.974947e+10,2.878621e+10,3.341477e+10,3.278038e+10,2.813031e+10,2.379054e+10,2.374147e+10,NaN
3,Albania,ALB,"Total reserves (includes gold, current US$)",FI.RES.TOTL.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.363792e+09,2.368942e+09,2.540674e+09,2.471403e+09,2.599864e+09,2.773278e+09,2.665216e+09,3.138518e+09,3.108607e+09,NaN
4,Andorra,AND,"Total reserves (includes gold, current US$)",FI.RES.TOTL.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
gold_dropped = gold[['Country Name', 'Country Code', '2016']]
gold_dropped.head()

,Country Name,Country Code,2016
0,Aruba,ABW,9.225509e+08
1,Afghanistan,AFG,7.281910e+09
2,Angola,AGO,2.374147e+10
3,Albania,ALB,3.108607e+09
4,Andorra,AND,NaN


In [6]:
gold_dropped.count()

Country Name    264
Country Code    264
2016            150
dtype: int64

In [7]:
gold_dropped.set_index('Country Code', inplace = True)
gold_dropped.head()

,Country Name,2016
Country Code,,
ABW,Aruba,9.225509e+08
AFG,Afghanistan,7.281910e+09
AGO,Angola,2.374147e+10
ALB,Albania,3.108607e+09
AND,Andorra,NaN


In [8]:
gold_dropped_sorted = gold_dropped.sort_values('2016', ascending=False).head(50)
gold_dropped_sorted.head()

,Country Name,2016
Country Code,,
CHN,China,3.097658e+12
JPN,Japan,1.216519e+12
CHE,Switzerland,6.789094e+11
SAU,Saudi Arabia,5.472606e+11
USA,United States,4.059423e+11


In [9]:
cities.head()

,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,CountryCode,ContinentName
0,Somaliland,Hargeisa,9.550000,44.050000,NaN,Africa
1,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000,GS,Antarctica
2,French Southern and Antarctic Lands,Port-aux-Français,-49.350000,70.216667,TF,Antarctica
3,Palestine,Jerusalem,31.766667,35.233333,PS,Asia
4,Aland Islands,Mariehamn,60.116667,19.900000,AX,Europe


In [10]:
cities_dropped = cities.drop('ContinentName',axis=1)

In [11]:
cities_dropped.head()

,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,CountryCode
0,Somaliland,Hargeisa,9.550000,44.050000,NaN
1,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000,GS
2,French Southern and Antarctic Lands,Port-aux-Français,-49.350000,70.216667,TF
3,Palestine,Jerusalem,31.766667,35.233333,PS
4,Aland Islands,Mariehamn,60.116667,19.900000,AX


In [12]:
combined = pd.merge(cities_dropped, gold_dropped_sorted, left_on='CountryName', right_on='Country Name')

In [13]:
combined.head(10)

,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,CountryCode,Country Name,2016
0,Algeria,Algiers,36.750000,3.050000,DZ,Algeria,1.207881e+11
1,Argentina,Buenos Aires,-34.583333,-58.666667,AR,Argentina,3.841451e+10
2,Australia,Canberra,-35.266667,149.133333,AU,Australia,5.503482e+10
3,Bangladesh,Dhaka,23.716667,90.400000,BD,Bangladesh,3.228384e+10
4,Brazil,Brasilia,-15.783333,-47.916667,BR,Brazil,3.649840e+11
5,Canada,Ottawa,45.416667,-75.700000,CA,Canada,8.271810e+10
6,Chile,Santiago,-33.450000,-70.666667,CL,Chile,4.049329e+10
7,China,Beijing,39.916667,116.383333,CN,China,3.097658e+12
8,Colombia,Bogota,4.600000,-74.083333,CO,Colombia,4.617377e+10
9,Czech Republic,Prague,50.083333,14.466667,CZ,Czech Republic,8.572530e+10


In [14]:
# Check if any countries are missing
combined.count()

CountryName         47
CapitalName         47
CapitalLatitude     47
CapitalLongitude    47
CountryCode         47
Country Name        47
2016                47
dtype: int64

In [15]:
combined.sort_values('2016', inplace=True)

In [16]:
combined.head(10)

,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,CountryCode,Country Name,2016
26,Morocco,Rabat,34.016667,-6.816667,MA,Morocco,2.535327e+10
13,Hungary,Budapest,47.500000,19.083333,HU,Hungary,2.582425e+10
20,Kazakhstan,Astana,51.166667,71.416667,KZ,Kazakhstan,2.942258e+10
28,Nigeria,Abuja,9.083333,7.533333,NG,Nigeria,3.002981e+10
33,Qatar,Doha,25.283333,51.533333,QA,Qatar,3.188735e+10
3,Bangladesh,Dhaka,23.716667,90.400000,BD,Bangladesh,3.228384e+10
21,Kuwait,Kuwait City,29.366667,47.966667,KW,Kuwait,3.393619e+10
27,Netherlands,Amsterdam,52.350000,4.916667,NL,Netherlands,3.590576e+10
46,Vietnam,Hanoi,21.033333,105.850000,VN,Vietnam,3.652729e+10
1,Argentina,Buenos Aires,-34.583333,-58.666667,AR,Argentina,3.841451e+10


In [17]:
gold_dropped_sorted

,Country Name,2016
Country Code,,
CHN,China,3.097658e+12
JPN,Japan,1.216519e+12
CHE,Switzerland,6.789094e+11
SAU,Saudi Arabia,5.472606e+11
USA,United States,4.059423e+11
HKG,"Hong Kong SAR, China",3.862405e+11
RUS,Russian Federation,3.770522e+11
KOR,"Korea, Rep.",3.701542e+11
BRA,Brazil,3.649840e+11


In [18]:
missing_countries = []
for index, row in gold_dropped_sorted.iterrows():
    if row['Country Name'] not in combined.values:
        missing_countries.append(row['Country Name'])
missing_countries # check which countries need to be fixed due to non-consistent naming

['Hong Kong SAR, China', 'Russian Federation', 'Korea, Rep.']

In [19]:
gold_dropped_sorted.loc[gold_dropped_sorted['Country Name'] == 'Russian Federation', 'Country Name'] = 'Russia'

In [20]:
missing_countries_corrected = ['Hong Kong', 'Russia', 'South Korea']
correction_dict = dict(zip(missing_countries, missing_countries_corrected))

In [21]:
for index, row in gold_dropped_sorted.iterrows():
    if row['Country Name'] in correction_dict.keys():
        gold_dropped_sorted.loc[index, 'Country Name'] = correction_dict[row['Country Name']]

In [22]:
combined = pd.merge(cities_dropped, gold_dropped_sorted, left_on='CountryName', right_on='Country Name')

In [23]:
combined

,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,CountryCode,Country Name,2016
0,Algeria,Algiers,36.750000,3.050000,DZ,Algeria,1.207881e+11
1,Argentina,Buenos Aires,-34.583333,-58.666667,AR,Argentina,3.841451e+10
2,Australia,Canberra,-35.266667,149.133333,AU,Australia,5.503482e+10
3,Bangladesh,Dhaka,23.716667,90.400000,BD,Bangladesh,3.228384e+10
4,Brazil,Brasilia,-15.783333,-47.916667,BR,Brazil,3.649840e+11
5,Canada,Ottawa,45.416667,-75.700000,CA,Canada,8.271810e+10
6,Chile,Santiago,-33.450000,-70.666667,CL,Chile,4.049329e+10
7,China,Beijing,39.916667,116.383333,CN,China,3.097658e+12
8,Colombia,Bogota,4.600000,-74.083333,CO,Colombia,4.617377e+10
9,Czech Republic,Prague,50.083333,14.466667,CZ,Czech Republic,8.572530e+10


In [24]:
# manually enter the data for Hong Kong
combined.loc[combined['CountryName'] == 'Hong Kong', 'CapitalName'] = 'Hong Kong'
combined.loc[combined['CountryName'] == 'Hong Kong', 'CapitalLatitude'] = 22.3964
combined.loc[combined['CountryName'] == 'Hong Kong', 'CapitalLongitude'] = 114.1095
combined.iloc[49]

CountryName          Hong Kong
CapitalName          Hong Kong
CapitalLatitude        22.3964
CapitalLongitude       114.109
CountryCode                 HK
Country Name         Hong Kong
2016                3.8624e+11
Name: 49, dtype: object

In [25]:
combined.drop(['Country Name'], inplace = True, axis=1)
combined.set_index('CountryCode', inplace=True)
combined.sort_values("2016", ascending=False, inplace=True)
combined.head()


,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,2016
CountryCode,,,,,
CN,China,Beijing,39.916667,116.383333,3.097658e+12
JP,Japan,Tokyo,35.683333,139.750000,1.216519e+12
CH,Switzerland,Bern,46.916667,7.466667,6.789094e+11
SA,Saudi Arabia,Riyadh,24.650000,46.700000,5.472606e+11
US,United States,Washington DC,38.883333,-77.000000,4.059423e+11


In [26]:
combined.to_csv('clean_capitals_gold.csv')

In [27]:
# make sure latitudes and logitudes are floats
combined.dtypes

CountryName          object
CapitalName          object
CapitalLatitude     float64
CapitalLongitude    float64
2016                float64
dtype: object

In [28]:
country_names = list(combined.loc[:, 'CountryName'])
country_names = [' '] + country_names

In [29]:
latitudes = list(combined.loc[:, 'CapitalLatitude'])

In [30]:
longitudes = list(combined.loc[:, 'CapitalLongitude'])

In [31]:
adjacency_matrix = [country_names]

In [32]:
lat_lon_pairs = list(zip(latitudes, longitudes))
lat_lon_pairs

[(39.916666666666657, 116.38333300000001),
 (35.68333333333333, 139.75),
 (46.916666666666657, 7.4666670000000002),
 (24.649999999999999, 46.700000000000003),
 (38.883333, -77.0),
 (22.3964, 114.1095),
 (55.75, 37.600000000000001),
 (37.549999999999997, 126.983333),
 (-15.783333333333333, -47.916666999999997),
 (28.600000000000001, 77.200000000000003),
 (1.2833333333333332, 103.84999999999999),
 (52.516666666666659, 13.4),
 (19.433333333333334, -99.133332999999993),
 (13.75, 100.516667),
 (48.866666666666667, 2.3333330000000001),
 (41.899999999999999, 12.483333),
 (51.5, -0.083333000000000004),
 (36.75, 3.0499999999999998),
 (-6.1666666666666679, 106.816667),
 (52.25, 21.0),
 (39.93333333333333, 32.866667),
 (31.766666666666666, 35.233333000000002),
 (3.1666666666666665, 101.7),
 (50.083333333333343, 14.466667000000001),
 (24.466666666666665, 54.366667000000007),
 (45.416666666666657, -75.700000000000003),
 (14.6, 120.96666699999999),
 (32.883333333333333, 13.166667000000002),
 (55.666

In [33]:
for index, pair1 in enumerate(lat_lon_pairs):
    temp = [country_names[index + 1]]
    for pair2 in lat_lon_pairs:
        temp.append(getDistance(pair1[0], pair1[1], pair2[0], pair2[1]))
    adjacency_matrix.append(temp)

In [34]:
adjacency_matrix

[[' ',
  'China',
  'Japan',
  'Switzerland',
  'Saudi Arabia',
  'United States',
  'Hong Kong',
  'Russia',
  'South Korea',
  'Brazil',
  'India',
  'Singapore',
  'Germany',
  'Mexico',
  'Thailand',
  'France',
  'Italy',
  'United Kingdom',
  'Algeria',
  'Indonesia',
  'Poland',
  'Turkey',
  'Israel',
  'Malaysia',
  'Czech Republic',
  'United Arab Emirates',
  'Canada',
  'Philippines',
  'Libya',
  'Denmark',
  'Spain',
  'Peru',
  'Norway',
  'Sweden',
  'Australia',
  'Lebanon',
  'South Africa',
  'Colombia',
  'Iraq',
  'Chile',
  'Romania',
  'Argentina',
  'Vietnam',
  'Netherlands',
  'Kuwait',
  'Bangladesh',
  'Qatar',
  'Nigeria',
  'Kazakhstan',
  'Hungary',
  'Morocco'],
 ['China',
  0.0,
  2099.756231672181,
  8074.2898099708555,
  6600.405468937408,
  11147.791595780838,
  1959.946706555792,
  5792.60033975077,
  955.679047747884,
  16930.427296981365,
  3779.5703324375036,
  4479.022757307709,
  7355.654827859621,
  12458.068192397046,
  3296.0604763059596,
  

In [35]:
# convert to be a numpy array for output
np_adjacency_matrix = np.array(adjacency_matrix)

In [36]:
# confirm the matrix has the right types and size
type(np_adjacency_matrix)
np_adjacency_matrix.shape

(51, 51)

In [37]:
np_adjacency_matrix

array([[' ', 'China', 'Japan', ..., 'Kazakhstan', 'Hungary', 'Morocco'],
       ['China', '0.0', '2099.756231672181', ..., '3652.452064144287',
        '7335.280521094176', '9938.27883482166'],
       ['Japan', '2099.756231672181', '0.0', ..., '5562.749448349543',
        '9047.167888851382', '11522.338529123877'],
       ..., 
       ['Kazakhstan', '3652.452064144287', '5562.749448349543', ..., '0.0',
        '3733.0804730661166', '6359.840786808368'],
       ['Hungary', '7335.280521094176', '9047.167888851382', ...,
        '3733.0804730661166', '0.0', '2626.9130696425145'],
       ['Morocco', '9938.27883482166', '11522.338529123877', ...,
        '6359.840786808368', '2626.9130696425145', '0.0']],
      dtype='<U20')

In [38]:
np.savetxt('distances.csv', np_adjacency_matrix, delimiter=',')

TypeError: Mismatch between array dtype ('<U20') and format specifier ('%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e')

In [41]:
with open('named_distances.csv', 'w') as f:
    for elt in adjacency_matrix:
        f.write(', '.join(str(v) for v in elt))
        f.write('\n')